In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path = '/content/drive/MyDrive/PlantDiseasesDataset.zip'

In [3]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/PlantDiseasesDataset.zip'
extract_dir = '/content/PlantDiseasesDataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Unzipped successfully to:", extract_dir)


Unzipped successfully to: /content/PlantDiseasesDataset


In [4]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

base_path = '/content/PlantDiseasesDataset'

plants = [p for p in os.listdir(base_path) if not p.startswith('.')]

print("Here are the plants you can pick:")
for i, plant in enumerate(plants):
    print(f"{i+1}. {plant}")

Here are the plants you can pick:
1. Grape
2. Peach
3. Strawberry
4. Cherry
5. Apple
6. Bell pepper
7. Corn
8. Citrus
9. Tomato
10. Potato


In [5]:
choice = input("Type the number of the plant you want: ")

try:
    choice = int(choice) - 1
    if choice < 0 or choice >= len(plants):
        print("Oops! That's not a valid number.")
        exit()
    selected_plant = plants[choice]
    print(f"You selected: {selected_plant}")
except:
    print("Please type a number!")
    exit()

Type the number of the plant you want: 3
You selected: Strawberry


In [6]:
train_path = os.path.join(base_path, selected_plant, 'train')
valid_path = os.path.join(base_path, selected_plant, 'valid')

In [7]:
height = 180
width = 180
channels = 3

input_shape = (height, width, channels)

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(height, width),
    batch_size=16,
    class_mode='sparse'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_path,
    target_size=(height, width),
    batch_size=16,
    class_mode='sparse'
)

Found 6146 images belonging to 2 classes.
Found 12 images belonging to 2 classes.


In [9]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=valid_generator
)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


385/385 ━━━━━━━━━━━━━━━━━━━━ 470s 1s/step - accuracy: 0.8410 - loss: 0.3923 - val_accuracy: 0.5833 - val_loss: 1.4085
Epoch 2/10
385/385 ━━━━━━━━━━━━━━━━━━━━ 470s 1s/step - accuracy: 0.9823 - loss: 0.0591 - val_accuracy: 0.5833 - val_loss: 1.3926
Epoch 3/10
385/385 ━━━━━━━━━━━━━━━━━━━━ 459s 1s/step - accuracy: 0.9904 - loss: 0.0337 - val_accuracy: 0.7500 - val_loss: 0.5678
Epoch 4/10
385/385 ━━━━━━━━━━━━━━━━━━━━ 457s 1s/step - accuracy: 0.9968 - loss: 0.0166 - val_accuracy: 0.8333 - val_loss: 0.7634
Epoch 5/10
385/385 ━━━━━━━━━━━━━━━━━━━━ 460s 1s/step - accuracy: 0.9972 - loss: 0.0139 - val_accuracy: 0.8333 - val_loss: 0.3248
Epoch 6/10
385/385 ━━━━━━━━━━━━━━━━━━━━ 455s 1s/step - accuracy: 0.9959 - loss: 0.0135 - val_accuracy: 0.8333 - val_loss: 0.3482
Epoch 7/10
385/385 ━━━━━━━━━━━━━━━━━━━━ 452s 1s/step - accuracy: 0.9989 - loss: 0.0042 - val_accuracy: 0.9167 - val_loss: 0.1072
Epoch 8/10
385/385 ━━━━━━━━━━━━━━━━━━━━ 466s 1s/step - accuracy: 0.9988 - loss: 0.0033 - val_accuracy: 0.833

In [ ]:
import matplotlib.pyplot as plt


plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np

Y_pred = model.predict(valid_generator)
y_pred = np.argmax(Y_pred, axis=1)

y_true = valid_generator.classes
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
print(classification_report(y_true, y_pred))